In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from urllib.error import HTTPError
import random
import time
from random import randint
from time import sleep

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}


In [2]:
def get_all_page_info():
    '''
    scrap all the comments and notes for insurer "Ag2r La Mondiale" for their product "Mutuelle santé".
    
    '''
    
    comment_gen = []
    comment_plus = []
    comment_moin = []
    comment_date = []
    Niveaux_des_prix = []
    Qualité_du_service_client = []
    Qualité_des_garanties = []
    Satisfaction = []

    
    # Get all the page urls 
    pages = []
    for i in range(1, 10):
        url = 'https://www.opinion-assurances.fr/assureur-ag2r-assurance-sante-page{}.html'.format(i)
        pages.append(url)
        
    # For every page in the interval 1 to 9
    
    for page in pages:
        try: 
            response = requests.get(page,headers=headers)
            # Pause the loop
            time.sleep(random.uniform(1.0, 2.0))
            print(page, response.status_code)
            print('----------------------------------------------------------------------------')
        except HTTPError as e:
            print(e)
            
        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        # Get all the urls of each page 
        
        internal_urls=[]
        for elem in page_html.find_all('a',attrs={"class" :"pull-right"}):
            # append all the urls in a list 
            internal_urls.append(elem.get('href'))
            
        # Build absolute urls
        
        absolute_urls =['https://www.opinion-assurances.fr'+ elem for elem in internal_urls if 'ag2r' in elem]
        
        
        #  crawl for every internal url
        
        for url in absolute_urls:
            time.sleep(random.uniform(1.0, 1.0))
        
            response = requests.get(url)
            if response.status_code != 200: return
            print(url, response.status_code)
            
            # Parse the html using beautiful soap and store in variable `internal_page_html`
            internal_page_html = BeautifulSoup(response.text, 'html.parser')
            
            
            # This block of code is use to scrap all the Commentaire général
            comment_g = internal_page_html.find('div',attrs={"class":"well break"})

            if comment_g is not None:
                comment_gen.append(comment_g.h2.text.strip())
                #print(comment_g.h2.text.strip())
            else:
                comment_gen.append("-")
                
            # This block of code is use scrap all the Commentaire “plus”
        
            comment_p = internal_page_html.find('div',attrs={"class":"plus"})
            if comment_p is not None:
                comment_plus.append(comment_p.h3.text.strip())
            else:
                comment_plus.append("-")
                
                
            # This block of code is use to scrap all the Commentaire “moins” 
            
            comment_m = internal_page_html.find('div',attrs={"class":"moins"})
        
            if comment_m is not None:
                comment_moin.append(comment_m.h3.text.strip())
            else:
                comment_moin.append("-")
                
                
            # This block of code is use to scrap all the Date du commentaire

       
            for com_date in internal_page_html.find_all('span',attrs={"id" :"MC_lbDate"}):
                comment_date.append(com_date.text.strip())
                
                
            # This block of code is use to extract all the Niveaux_des_prix 
        
        
            niveaux_prix = internal_page_html.find_all('div',attrs={"class" :"alf_stackLevel alf_stackLevelReadOnly"})[0]
            niveaux_prix_rating= niveaux_prix.find('div',attrs={"class" :"alf_label"})
            Niveaux_des_prix.append(niveaux_prix_rating.text.strip()) 

            # This block of code is use to extract all the Qualité du service client

            service_client = internal_page_html.find_all('div',attrs={"class" :"alf_stackLevel alf_stackLevelReadOnly"})[1]
            service_client_rating = service_client.find('div',attrs={"class" :"alf_label"})
            Qualité_du_service_client.append(service_client_rating.text.strip())


             # This blcok of code is use to extract all the Qualité_des_garanties 

            qualite_garanties = internal_page_html.find_all('div',attrs={"class" :"alf_stackLevel alf_stackLevelReadOnly"})[2]
            qualite_garanties_rating = qualite_garanties.find('div',attrs={"class" :"alf_label"})
            Qualité_des_garanties.append(qualite_garanties_rating.text.strip())


             # This blcok of code is use to extract all the Satisfaction

            satisfaction= internal_page_html.find_all('div',attrs={"class" :"alf_stackLevel alf_stackLevelReadOnly"})[3]
            satisfaction_rating= satisfaction.find('div',attrs={"class" :"alf_label"})
            Satisfaction.append(satisfaction_rating.text.strip())
            
    #  Verfiy the length of the list 
    
    print(len(comment_gen),len(comment_plus),len(comment_moin),len(comment_date),len(Niveaux_des_prix), len(Qualité_du_service_client),len(Satisfaction), len(Qualité_des_garanties))
    
    # Create a python dictinary 
    page_info = {
        'com_general':comment_gen,
        'com_plus': comment_plus,
        'com_moin': comment_moin,
        'date': comment_date,
        "niveaux des prix":Niveaux_des_prix,
        "qualité_du_service_client": Qualité_du_service_client,
        "qualité_des_garanties":Qualité_des_garanties,
        "satisfaction":Satisfaction,
        "assureur": "Ag2r La Mondiale",
        "produit": "vie"
    }
    
    return page_info 


        


In [3]:
def writeToExcel(data):
    '''
    Take the dict of results and write to an excel file.

    '''
    
    data_frame = pd.DataFrame(data)
    data_frame.to_excel(r'/home/tashitsering/Desktop/PMP_test_tashi/data/output.xlsx')


In [4]:
if __name__ == "__main__": 
    all_data = get_all_page_info()
    writeToExcel(all_data)


https://www.opinion-assurances.fr/assureur-ag2r-assurance-sante-page1.html 200
----------------------------------------------------------------------------
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-102184.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101844.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101814.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101798.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101632.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101553.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101188.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101123.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101120.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-101040.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-100828.html 200
https://

https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-84394.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-84223.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-83637.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-83369.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-83177.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-83171.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-82963.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-82947.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-82749.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-82694.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-82669.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-82598.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-82435.html 200
https://www.opinion-assur

https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-66590.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-66250.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-66193.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-65277.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-65261.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-65159.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-64972.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-64861.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-64827.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-64730.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-64401.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-64371.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-64160.html 200
https://www.opinion-assur

https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-10941.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-10125.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-10044.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-9471.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-8815.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-8791.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-8496.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-8290.html 200
https://www.opinion-assurances.fr/assureur-ag2r-assurance-sante-page9.html 200
----------------------------------------------------------------------------
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-7469.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-7360.html 200
https://www.opinion-assurances.fr/ag2r-assurance-sante-avis-7285.html 200
https://www.opinion-assuran